# Golf Swing Part III- Using pre-trained models and FiftyOne to find golf positions
> Using Python and Coco to identify parts of the body and club during a golf swing 

- toc: true 
- badges: true
- comments: true
- categories: [Neural networks,FiftyOne, , Golf Swing, Python]

![](ghtop_images/header2.png)

## Overview

In a previous part [Part 1](https://thomashsimm.com/jupyter/2021/12/01/GolfPos1FastAI.html) a neural network model was used to find positions on the body during a golf swing. The model was not particularly succesful, perhaps due to the lack of data (specific to the golf swing) that was used to train the model on.

This problem can be got around by using a model that has been pre-trained on human gestures. Several pre-trained models can be found here [Pre-trained models](https://voxel51.com/docs/fiftyone/user_guide/model_zoo/models.html). I tried a few and found the chose the model `keypoint-rcnn-resnet50-fpn-coco-torch` worked well with this data. [Link to model](https://voxel51.com/docs/fiftyone/user_guide/model_zoo/models.html#keypoint-rcnn-resnet50-fpn-coco-torch) and [Paper of model](https://arxiv.org/abs/1603.00502).


More information on [fiftyOne](https://voxel51.com/).


The input to the model is taken from [Part 2](https://thomashsimm.com/neural%20networks/pytourch/golf%20swing/python/2022/02/26/GolfSwingPart2.html) which separated a golf video into a series of images of the swing.

## Code

The code can be run on google colab here [COCO50_1](https://colab.research.google.com/drive/17Bb8n9D9_4aAuJlCf2yOQE08L7vjUtBm?usp=sharing) (works best on google chrome)

## Installs and imports

In [ ]:
!pip uninstall opencv_python_headless

!pip install opencv-python-headless==4.5.4.60

!pip install fiftyone

import fiftyone as fo

## Upoad some images to the workspace

Untar and create a dataset object from them

And look at them

The image files can be found here [GC2.tgz](https://drive.google.com/file/d/1FMOP3Lm-0Ed1SmuPYdcm6emUu-Jhv2CW/view?usp=sharing)

In [ ]:
import tarfile
my_tar = tarfile.open('/content/GC2.tgz')
my_tar.extractall('/content/my_folder') # specify which folder to extract to
my_tar.close()

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

import fiftyone as fo

name = "my_folder"
dataset_dir = "/content"

# Create the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fo.types.ImageDirectory,
    name=name,
)

session = fo.launch_app(dataset)

![](ghtop_images/fiftyOneScreen.png)


This screen is interative and allows us to look at the images

## Load the trained model 

Apply the model to the dataset

View the results

In [ ]:
model = foz.load_zoo_model("keypoint-rcnn-resnet50-fpn-coco-torch")

# label_types=["classification", "classifications", "detections", "instances", "segmentations", "keypoints", "polylines", "polygons", "scalar"],

dataset.apply_model(model, label_field="predictions",label_types='predictions_keypoints')

session = fo.launch_app(dataset)

![](ghtop_images/foo2.png)

> youtube: https://youtu.be/dkxtOBWD7Vw

## Extract data from the model

We might want to use the data from the model outside of fiftyOne. 

In the following I extract the data so that it can be plotted.

In [ ]:
def plotPredOne(i):
  import numpy as np

  import matplotlib.pyplot as plt
  import matplotlib.image as mpimg

  img = mpimg.imread(i['filepath'])

  #need to take account of more than one person in image
  points1 = np.array(i['predictions_keypoints']['keypoints'][0]['points'])
  adjPts = np.shape(img)[0]
  box1 = np.array(i['predictions_detections']['detections'][0]['bounding_box']) 
  box1=box1*adjPts
  # Bboxes are in [top-left-x, top-left-y, width, height] format
  box2=np.array([ 
      [box1[0], box1[1]],
      [box1[0] +box1[2] ,box1[1] ],
      [box1[0] +box1[2] ,box1[1] +box1[3]] ,
      [box1[0]  ,box1[1] +box1[3]],
      [box1[0], box1[1]]
      ])
 
  plt.figure()
  plt.imshow(img)

  plt.plot(points1[:,0]*adjPts,points1[:,1]*adjPts, '+k',markersize=10,linewidth=3)
  plt.plot(box2[:,0],box2[:,1], '--og',markersize=10,linewidth=3)

    #back of body
  v=[4,6,12,14,16]
  plt.plot(points1[v,0]*adjPts,points1[v,1]*adjPts, '-k<',markersize=10,linewidth=2)

  #front of body
  v=[0,5,11,13,15]
  plt.plot(points1[v,0]*adjPts,points1[v,1]*adjPts, '-w>',markersize=10,linewidth=2)

  vects = np.array([[ 5,6],#shoulders also 4?
         [11,12], #hips
         [13,14], #knees
         [15,16],#heels
         [7,8],#elbows
         [9,10],#hands
         ]) 
  mak='gcyrmb'
  for iv,v in enumerate(vects):
    plt.plot(points1[v,0]*adjPts,points1[v,1]*adjPts, '-'+mak[iv],markersize=10,linewidth=3)


In [ ]:
for iii,i in enumerate(dataset):
  
  plotPredOne(i)


![](ghtop_images/foo3.png)